In [28]:
import scipy.io as sio
from scipy.signal import resample
import numpy as np
def window_eeg(x, window_sec, stride_sec, data_len_sec, samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    nwindow = int((data_len_sec - window_sec)/stride_sec + 1)
    #print(nwindow)
    window_pt = int(samp_freq*window_sec)
    windowed_eeg = np.zeros((nwindow,window_pt,nchannel))
    for i in range(nwindow):
        x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
        #print(x_windowed.shape)
        x_windowed = np.transpose(x_windowed)
        windowed_eeg[i,:,:] = x_windowed
    return windowed_eeg
def down_sample(x, samp_freq, target_freq):
    nchannel = x.shape[0]
    original_len = x.shape[1]
    target_len = int(original_len*target_freq/samp_freq)
    target_x = np.zeros((nchannel,target_len))
    for i in range(nchannel):
        target_x[i,:] = resample(x[i,:], target_len)
    return target_x
window = 60
stride = window
target_freq = 200
data_len_sec = 600
nwindow = int((data_len_sec - window)/stride + 1)

In [29]:
from numpy.random import choice
subject = 'Dog_3'
npreictal = 12
ninterictal = 240
preictal_idx = [i for i in range(npreictal)]
interictal_idx = [i for i in range(ninterictal)]
n_preictal_train = 11
n_interictal_train = int(n_preictal_train*2)
train_preictal_idx = choice(preictal_idx, n_preictal_train, replace = False)
train_preictal_mat_idx = []
for i in train_preictal_idx:
    for j in np.arange(i*6,i*6+6):
        train_preictal_mat_idx.append(j)
train_interictal_idx = choice(interictal_idx, n_interictal_train, replace = False)
train_interictal_mat_idx = []
for i in train_interictal_idx:
    for j in np.arange(i*6,i*6+6):
        train_interictal_mat_idx.append(j)
test_preictal_idx = np.setdiff1d(preictal_idx, train_preictal_idx)
test_interictal_idx = np.setdiff1d(interictal_idx, train_interictal_idx)
test_interictal_idx = choice(test_interictal_idx, int((npreictal-n_preictal_train)*2), replace = False)
test_preictal_mat_idx = []
for i in test_preictal_idx:
    for j in np.arange(i*6,i*6+6):
        test_preictal_mat_idx.append(j)
test_interictal_mat_idx = []
for i in test_interictal_idx:
    for j in np.arange(i*6,i*6+6):
        test_interictal_mat_idx.append(j)        

In [32]:
X_train = []
Y_train = []
for i in train_interictal_mat_idx:
    filename = '{0}\\{1}_interictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
    file = sio.loadmat(filename)
    key = 'interictal_segment_{0}'.format(str(i+1)) 
    x = file[key]['data'][0,0]
    data_len_sec = file[key]['data_length_sec'][0,0][0,0]
    samp_freq = file[key]['sampling_frequency'][0,0][0,0]
    x_down = down_sample(x, samp_freq, target_freq)
    x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
    for j in range(x_windowed.shape[0]):
        X_train.append(x_windowed[j,:,:])
        Y_train.append(0)
for i in train_preictal_mat_idx:
    filename = '{0}\\{1}_preictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
    file = sio.loadmat(filename)
    key = 'preictal_segment_{0}'.format(str(i+1)) 
    x = file[key]['data'][0,0]
    data_len_sec = file[key]['data_length_sec'][0,0][0,0]
    samp_freq = file[key]['sampling_frequency'][0,0][0,0]
    x_down = down_sample(x, samp_freq, target_freq)
    x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
    for j in range(x_windowed.shape[0]):
        X_train.append(x_windowed[j,:,:])
        Y_train.append(1)
X_test = []
Y_test = []
for i in test_interictal_mat_idx:
    filename = '{0}\\{1}_interictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
    file = sio.loadmat(filename)
    key = 'interictal_segment_{0}'.format(str(i+1)) 
    x = file[key]['data'][0,0]
    data_len_sec = file[key]['data_length_sec'][0,0][0,0]
    samp_freq = file[key]['sampling_frequency'][0,0][0,0]
    x_down = down_sample(x, samp_freq, target_freq)
    x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
    for j in range(x_windowed.shape[0]):
        X_test.append(x_windowed[j,:,:])
        Y_test.append(0)
for i in test_preictal_mat_idx:
    filename = '{0}\\{1}_preictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
    file = sio.loadmat(filename)
    key = 'preictal_segment_{0}'.format(str(i+1)) 
    x = file[key]['data'][0,0]
    data_len_sec = file[key]['data_length_sec'][0,0][0,0]
    samp_freq = file[key]['sampling_frequency'][0,0][0,0]
    x_down = down_sample(x, samp_freq, target_freq)
    x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
    for j in range(x_windowed.shape[0]):
        X_test.append(x_windowed[j,:,:])
        Y_test.append(1)



In [31]:
test_preictal_mat_idx

[42, 43, 44, 45, 46, 47]

In [36]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [37]:
import pickle
processed_data = {'X_train':X_train, 'Y_train':Y_train, 'X_test': X_test, 'Y_test': Y_test}
pickle.dump(processed_data, open( "Dog_3_60s_60s_11_1.p", "wb"), protocol=4)

In [38]:
X_train.shape

(1980, 12000, 16)